In [ ]:
import os
import random
import matplotlib.pyplot as plt
from PIL import Image
from skimage import io, filters

In [ ]:
# Set the path to your image folders
base_path = 'poliamid_clusters/data/images'
output_path = 'poliamid_clusters/combined_images.png'  # Path to save the output image

# Get all subdirectories (folders)
subfolders = [f.path for f in os.scandir(base_path) if f.is_dir()]

# Set up the plot
num_subfolders = len(subfolders)
plt.figure(figsize=(12, 3 * num_subfolders))  # Adjust figure size based on number of folders

for i, folder in enumerate(subfolders, 1):
    # Get all image files in the folder
    image_files = [f for f in os.listdir(folder) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]
    
    if len(image_files) < 2:
        print(f"Folder {folder} has less than 2 images. Skipping...")
        continue
    
    # Randomly select 2 images
    selected_images = random.sample(image_files, 2)
    
    # Plot the images
    for j, img_file in enumerate(selected_images, 1):
        img_path = os.path.join(folder, img_file)
        img = Image.open(img_path)
        
        plt.subplot(num_subfolders, 2, (i-1)*2 + j)
        plt.imshow(img)
        plt.title(f"Folder: {os.path.basename(folder)}\nImage: {img_file}")
        plt.axis('off')

plt.tight_layout()
plt.savefig(output_path, bbox_inches='tight', dpi=300)  # Save to file

In [ ]:
output_path = 'poliamid_clusters/combined_images.png'  

subfolders = [f.path for f in os.scandir(base_path) if f.is_dir()]
subfolders

In [ ]:
# image_path = 'poliamid_clusters/data/images/Полиамид 12   1 кадр в мин   (3-10-2024_18-11--6-46)/IMG_1259.JPG'
# image_path = 'poliamid_clusters/data/images/Полиамид 12   1 кадр в мин   (3-10-2024_18-11--6-46)/IMG_2266.JPG'
# image_path = 'poliamid_clusters/data/images/Полиамид 12   1 кадр в мин   (24-09-2024_12-27--1-0)/IMG_6186.JPG'
# image_path = 'poliamid_clusters/data/images/Полиамид 12   1 кадр в мин   (24-09-2024_12-27--1-0)/IMG_5589.JPG'
# image_path = 'poliamid_clusters/data/images/Полиамид 12   1 кадр в мин   (12-09-2024_12-26--1-3)/IMG_4097.JPG'
# image_path = 'poliamid_clusters/data/images/Полиамид 12   1 кадр в мин   (12-09-2024_12-26--1-3)/IMG_3603.JPG'
# image_path = 'poliamid_clusters/data/images/Полиамид 12   1 кадр в мин   (1-10-2024_14-6--2-40)/IMG_0585.JPG'
# image_path = 'poliamid_clusters/data/images/Полиамид 12   1 кадр в мин   (1-10-2024_14-6--2-40)/IMG_0268.JPG'
# image_path = 'poliamid_clusters/data/images/Полиамид 12   1 кадр в мин   (09-09-2024_12-10--0-44)/IMG_9009.JPG'
# image_path = 'poliamid_clusters/data/images/Полиамид 12   1 кадр в мин   (09-09-2024_12-10--0-44)/IMG_8955.JPG'
# image_path = 'poliamid_clusters/data/images/Полиамид 12   1 кадр в мин   (02-09-2024_13-33--2-7)/IMG_8747.JPG'
# image_path = 'poliamid_clusters/data/images/Полиамид 12   1 кадр в мин   (02-09-2024_13-33--2-7)/IMG_6932.JPG'

# img = io.imread(image_path)
# img = img[:,900:-900]

# plt.imshow(img)
# plt.show()

# Image preprocess

In [ ]:
image_path = 'poliamid_clusters/data/images/Полиамид 12   1 кадр в мин   (09-09-2024_12-10--0-44)/IMG_8955.JPG'
# image_path = 'poliamid_clusters/data/images/Полиамид 12   1 кадр в мин   (02-09-2024_13-33--2-7)/IMG_8747.JPG'

img = io.imread(image_path)
img = img[500:-300,1200:-1500]

img=img[:2000,:2000]
img[:250,:250]=np.full((250,250,3),20)

plt.imshow(img)
plt.show()

In [ ]:

thresh = filters.threshold_otsu(img)
binary = img > thresh  # Apply threshold
binary = binary*255


plt.imshow(binary)
plt.show()

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress


def find_contours(binary_image):
    """Find contours using OpenCV"""
    contours, _ = cv2.findContours(binary_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def draw_contours(original_img, contours):
    """Draw contours on the original image"""
    contour_img = cv2.cvtColor(original_img, cv2.COLOR_GRAY2BGR)
    cv2.drawContours(contour_img, contours, -1, (0, 255, 0), 2)
    return contour_img

def fractal_dimension_box_counting(binary_image, plot=False):
    """Calculate fractal dimension using box-counting method"""
    def count_boxes(image, box_size):
        boxes = image.shape[0] // box_size
        count = 0
        for i in range(boxes):
            for j in range(boxes):
                if np.any(image[i*box_size:(i+1)*box_size, j*box_size:(j+1)*box_size]):
                    count += 1
        return count
    
    sizes = 2**np.arange(1, 8)
    counts = []
    
    for size in sizes:
        counts.append(count_boxes(binary_image, size))
    
    # Linear regression in log-log space
    log_sizes = np.log(sizes)
    log_counts = np.log(counts)
    slope, intercept, r_value, _, _ = linregress(log_sizes, log_counts)
    
    if plot:
        plt.figure(figsize=(8, 4))
        plt.plot(log_sizes, log_counts, 'bo-')
        plt.plot(log_sizes, intercept + slope*log_sizes, 'r--')
        plt.title(f'Fractal Dimension: {-slope:.3f} (R²={r_value**2:.3f})')
        plt.xlabel('log(Box Size)')
        plt.ylabel('log(Box Count)')
        plt.grid()
        plt.show()
    
    return -slope

# Main processing
image_path = 'poliamid_clusters/data/images/Полиамид 12   1 кадр в мин   (09-09-2024_12-10--0-44)/IMG_8955.JPG'


original_img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

original_img = original_img[500:-300,1200:-1500]
original_img=original_img[:2000,:2000]

# blurred = cv2.GaussianBlur(img, (5, 5), 0)
_, binary = cv2.threshold(original_img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
contours = find_contours(binary)

# 2. Create contour visualization
contour_img = draw_contours(original_img, contours)

fig, axes = plt.subplots(1,2)

axes[0].imshow(original_img, cmap='gray')
axes[1].imshow(contour_img)

In [ ]:
len_list=[]

for contour in contours:
    len_list.append(len(contour))

plt.hist(len_list,bins=1000)
plt.xlim(0,200)
# plt.yscale('log')
plt.xlabel('number of points in each contour ')
plt.title('Histogram of contour sizes')

In [ ]:
for i, contour in enumerate(contours):
    # Create a blank image with just this contour
    if len(contours)>10:
        mask = np.zeros_like(binary)
        cv2.drawContours(mask, [contour], -1, 255, thickness=cv2.FILLED)
        
        # Calculate fractal dimension
        fd = fractal_dimension_box_counting(mask, plot=False)
        print(f"Contour {i+1}: Fractal Dimension = {fd:.3f}")

# Display results
plt.figure(figsize=(12, 6))
plt.subplot(121), plt.imshow(cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB))
plt.title('Original Image'), plt.axis('off')
plt.subplot(122), plt.imshow(cv2.cvtColor(contour_img, cv2.COLOR_BGR2RGB))
plt.title(f'Detected Contours ({len(contours)} found)'), plt.axis('off')
plt.show()

In [ ]:
plt.imshow(contour_img)